# Multivariate time series visualization, analysis and forecasting using ML algorithms   

**Purpose** : This automation is a modularized toolkit for multivariate time series analysis and forecasting using Prophet, Random Forest and Vector Autoregression algorithms. Multivariate time series has more than one time-dependent variable which are correlated with each other and can be used to forecast future values. The user can compare forecast accuracies and results from the three algorithms and choose which output to proceed with. 
The notebook is divided into the following sections:

**Edit Date** : 12/2/2020

- ##### **Algorithms Included in the notebook**
  - Random Forest
  - Vector Autoregression
  - Prophet
</br></br> 

- ##### **Sections in the Notebook**
  1. Importing libraries
  2. Data preparation
  3. Visual analysis of time series
  4. Forecasting algorithms - Random forest, Vector Autoregression, Prophet
  5. Comparison of error metrics for the models
  6. Saving forecast results
  
- ##### **Data**
  - Mocked Data 
</br></br>


- **Goal**
    - Predict the price of a home.
    

Some code cells have an option to toggle between show/hide code. This is for the cells for which no user input is required. Hence, after running the cells, the user may choose to hide the code to maintain simplicity of the notebook. 

# Dependencies

Load the dependencies for model development. Current package requirements include:
* Sklearn
* Pandas
* Numpy
* Matplotlib
* fbprophet
* statsmodels

The current template uses the following versions:
* Sklearn version: 0.21.2
* Pandas version: 1.1.2
* Numpy version: 1.16.4
* Matplotlib version: 3.1.0
* FBProphet version: 0.7.1
* Statsmodels version: 0.10.0


## Reference(s)

* [Forecasting using Random Forest](https://medium.com/@oemer.aslantas/forecasting-sales-units-with-random-forest-regression-on-python-a75d92910b46)

* [Hyperparameter tuning in Random Forest](https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74)

* [Vector Autoregression (VAR) to forecast demand](https://www.machinelearningplus.com/time-series/vector-autoregression-examplespython/#:~:text=Vector%20Autoregression%20(VAR)%20is%20a%20multivariate%20forecasting%20algorithm%20that%20is,series%20should%20influence%20each%20other.)

* [Time Series forecasting using Facebook's Prophet](https://towardsdatascience.com/time-series-prediction-using-prophet-in-python-35d65f626236)

__Recommended__ : Use this notebook after cleaning the data for missing values / outliers

# 1.1. Importing libraries

In [18]:
## To install libraries uncomment the following lines

import sys
!{sys.executable} -m pip install numpy cython matplotlib scipy pandas pystan==2.18.0.0 fbprophet --user

In [16]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    import imp
import pip
import pandas as pd
import datetime
import numpy as np
import pylab
import matplotlib
import sklearn
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split 
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tsa.stattools import acf
from sklearn.metrics import mean_absolute_error, mean_squared_error  ## For calculation of error metrics
import math   
from IPython.display import display, Markdown, Latex
from scipy import stats
from pandas.plotting import autocorrelation_plot
from sklearn.model_selection import GridSearchCV
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from fbprophet import Prophet
import fbprophet
import statsmodels

In [17]:
print ("Sklearn version: "+sklearn.__version__)
print ("Pandas version: "+pd.__version__)
print ("Numpy version: "+np.__version__)
print ("Matplotlib version: "+matplotlib.__version__)
print ("FBProphet version: "+fbprophet.__version__)
print ("Statsmodels version: "+statsmodels.__version__)

Sklearn version: 0.21.2
Pandas version: 1.1.2
Numpy version: 1.16.4
Matplotlib version: 3.1.0
FBProphet version: 0.7.1
Statsmodels version: 0.10.0


# Function to create a toggle hide/show option for each cell
The following function will help hide code cells where a user interaction is not required in a particular cell

In [ ]:
#### function for hide/show cell toggle

from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

# 1.2. Data reading

Read in a clean and processed csv file for the modelling. This activity will heavily depend on your specific problem and the data set that you are using. User should make sure continuation in time frames used in the data and clean for any outliers that might account to some anomalous events. This can enhance the performance of the model but this is not a necessary step.


In [ ]:
## Function to read the file name from widget
def file_path_func(file):
    global file_path
    
    file_path = file
    
hide_toggle()

**Enter csv file path in the below cell.**

Type in the file name in the widget below followed by '.csv' format.

In [ ]:
interact(file_path_func, file=widgets.Text(description='Enter file path', disabled=False))

In [ ]:
# Reading the data and creating a list of column names 
df_train = pd.read_csv(file_path)

cols_list = []
for i in range(len(df_train.columns)):
    cols_list.append(df_train.columns[i])
cols_list.insert(0,'Select')

hide_toggle()

# Important Functions used in the notebook

The following functions are used to get user inputs in various widgets used through out the notebook

In [ ]:
## Important Functions used in the notebook
## To select date column
def date_func(date):
    global date_col
    if date=='Select':
        return
    else:
        date_col = date
        
        
## To select the target column
def target_var_func(target):
    global target_var
    if target == 'Select':
        return
    else:
        target_var = target

        
## To select the forecasting level columns
def forecast_level_func(level):
    global forecast_level
    if len(level) == 0:
        return
    else:
        forecast_level = list(level)


## To select the date column formats
def date_format_func(format1):
    global date_format
    if format1=='Select':
        return
    else:
        date_format = format1

available_formats = ['%d/%m/%Y','%d-%m-%Y','%m/%d/%Y','%m-%d-%Y','%Y/%d/%m',\
                     '%Y-%d-%m','%Y/%d/%m','%Y-%d-%m']
available_formats.insert(0,'Select')

hide_toggle()

# Functions for Error metrics

The following block gives a function to calculate the error metrics that are used to measure and compare the performance of the algorithms. MAPE is used as an industry wide metric used to assess the accuracy of diffrent time series models. We will also be looking at sMAPE,MSE and RMSE to compare the algorithms. The following refernces will help understand which secanrios each metric is most helpful:

 * [MAPE vs sMAPE](https://towardsdatascience.com/choosing-the-correct-error-metric-mape-vs-smape-5328dec53fac)
 * [Forecasting KPIs](https://towardsdatascience.com/forecast-kpi-rmse-mae-mape-bias-cdc5703d242d)

In [ ]:
## Functions for calculating error metrics
def mean_absolute_percentage_error(y_true, y_pred):
    """
    Computes MAPE values given the actuals and forecasted numbers
    
    Args:
        y_true (list): Actual numbers
        y_pred (list): Predicted numbers
    
    Returns:
        float: MAPE value
        float: Minimum Absolute Error Percentage
        float: Maximum Absolute Error Percentage
        
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    ae = np.abs((y_true - y_pred) / y_true)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100 , min(ae)*100, max(ae)*100


def smape(y_true, y_pred):
    """
    Computes the sMAPE value given the actuals and forecasted numbers
        
    Args:
        y_true (list): Actual numbers
        y_pred (list): Predicted numbers
    
    Returns:
        float: sMAPE value
        
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))


def get_error_metrics(y_true,y_pred,m_string):
    """
    Computes and stores errors metrics in a dictionary
    
    Args:
        y_true (list): Actual numbers
        y_pred (list): Predicted numbers
        m_string : Model Name
        
    Returns:
        pd.DataFrame: Error metrics
        pd.DataFrame: MAPE metrics
        
    """
    mape,min_val,max_val =  mean_absolute_percentage_error(y_true,y_pred)
    error_table_var = pd.DataFrame({'Model_Name':m_string,
                                        'MAPE':mape,
                                        'MSE': mean_squared_error(y_true, y_pred),
                                        'RMSE': math.sqrt(mean_squared_error(y_true, y_pred)),
                                        'SMAPE': smape(y_true, y_pred)}, index=[0])
    return error_table_var

hide_toggle()

**Select date column**

Please select the date column from the following widget

In [ ]:
interact(date_func, date=widgets.Dropdown(options=cols_list,value='Select',description='Date column',disabled=False))

**Select Appropriate Date Format (single option)**

**Example: %Y/%m/%d means "YYYY/MM/DD"
%Y-%m-%d means "YYYY-MM-DD"**

In [ ]:
interact(date_format_func, format1=widgets.Dropdown(options=available_formats,value='Select',description='Date column Format',disabled=False))

**Select forecast column**

Select the column for which you want to forecast the values

In [ ]:
interact(target_var_func, target=widgets.Dropdown(options=cols_list,value='Select',description='Forecast column',disabled=False))

**Select forecast level (multiple option)**

Select the level of time series. This can be used in case you have a panel data at two or more different levels. The following link will help undesrtand a panel data.

 * [What is a Panel Data](https://en.wikipedia.org/wiki/Panel_data#:~:text=Panel%20data%20is%20a%20subset,time%20point%20for%20the%20latter)

In [ ]:
interact(forecast_level_func, level=widgets.SelectMultiple(options=df_train.columns, description='Forecast level', disabled=False))

# Data Sanity Check

A time series data should ideally consist of equally spaced time points. The following code checks whether the time gaps between the data point are consistent. It is highly recommended to use a data with conistent time point only. 

In [ ]:
#### calculating date lag

df_train[date_col] = pd.to_datetime(df_train[date_col], format = date_format)
df_train['date_lag'] = df_train.groupby(forecast_level)[date_col].shift(1)
df_train['date_delta'] = df_train[date_col] - df_train['date_lag']
df_train['date_delta'] = df_train['date_delta'].dt.days

hide_toggle()

# Data Cleaning and Summary

A best practice for any modelling steps should be to visualize the data and perform a rigorous EDA(Exploratory Data Analysis). Here we view a very basic set of summary. An overview of the steps involved in cleaning and basic EDA of time series data is as follows:
 * Time components and outlier identification
 * Summary statistics at different slices of the data, visualize the distribution and spread of data points
 * Feature Engineering : A classical approach of faeture engineering to transform time series forecasting problems into supervised learning problems is to create lagged features.
 * Visualizing the time series : Plot the data to view histograms, ine plot, lagged scatter plots, box-whiskers plot etc.
 * Power Transformations
 * Resampling the time series : Interpolate data point to change the frequency of time point in business needs , eg : if daily forecasts required but weekly data present
 * Explore the temporal structure of the data including the predictive piwer of the data

In [ ]:
#### summarizing dataset

df1 = pd.DataFrame(df_train.groupby(forecast_level)[date_col].min()).rename(columns={date_col:'min_date'})
df2 = pd.DataFrame(df_train.groupby(forecast_level)[date_col].max()).rename(columns={date_col:'max_date'})
df3 = pd.DataFrame(df_train.groupby(forecast_level)[date_col].count()).rename(columns={date_col:'count_date'})
df4 = pd.DataFrame(df_train.groupby(forecast_level)[target_var].min()).rename(columns={target_var:'min_forecast_col'})
df5 = pd.DataFrame(df_train.groupby(forecast_level)[target_var].max()).rename(columns={target_var:'max_forecast_col'})
df6 = pd.DataFrame(df_train.groupby(forecast_level)[target_var].mean()).rename(columns={target_var:'mean_forecast_col'})
df7 = pd.DataFrame(df_train.groupby(forecast_level)['date_delta'].unique()).rename(columns = {'date_delta':'distinct_day_gaps'})

df_concat = pd.concat([df1,df2,df3,df4,df5,df6,df7], axis=1)

hide_toggle()

In [ ]:
print('\033[1m' + 'The following is the summary of your input dataset for the first 5 rows.' + '\033[0m')
df_concat.head()

In [ ]:
#### printing dataframe with >2 distinct day gaps

df_short = pd.DataFrame()

for i in range(df_concat.shape[0]):
    if len(df_concat.iloc[i]['distinct_day_gaps'])>2:
        df_filter = pd.DataFrame(df_concat.iloc[i]).transpose().reset_index()
        df_short = df_short.append(df_filter)
        
hide_toggle()

In [ ]:
print('There are','\033[1m',df_short.shape[0],'\033[0m','entries with more than 2 distinct day gaps.')
if df_short.shape[0] != 0:
    print('Error message: Please ensure consistency in lag in date column.')
    
hide_toggle()

# 1.3. Visual analysis of time series

##### Time Series Decomposition : 
Time series decomposition involves thinking of a series as a combination of level, trend, seasonality, and noise components. Decomposition provides a useful abstract model for thinking about time series generally and for better understanding problems during time series analysis and forecasting.
A given time series is thought to consist of three systematic components including level, trend, seasonality, and one non-systematic component called noise.

* **Level**: The average value in the series.
* **Trend**: The increasing or decreasing value in the series.
* **Seasonality**: The repeating short-term cycle in the series.
* **Noise**: The random variation in the series

##### Difference between a multiplicative and an additive time series.

In a multiplicative time series, the components multiply together to make the time series. If you have an increasing trend, the amplitude of seasonal activity increases. Everything becomes more exaggerated. This is common when you’re looking at web traffic.

In an additive time series, the components add together to make the time series. If you have an increasing trend, you still see roughly the same size peaks and troughs throughout the time series. This is often seen in indexed time series where the absolute value is growing but changes stay relative.

Here we consider an additive time series. User input required if a different case encountered.

In [ ]:
if len(forecast_level)==2:
    
    level1_col = list(df_train[forecast_level[0]].unique())
    level1_col.insert(0,'Select')
    
    level2_col = list(df_train[forecast_level[1]].unique())
    level2_col.insert(0,'Select')
    
    def level1_func(l1):
        global level1
        if l1 == 'Select':
            return
        else:
            level1 = l1
        
    def level2_func(l2):
        global level2
        if l2 == 'Select':
            return
        else:
            level2=l2
            
if len(forecast_level) == 1:
    
    level_col = list(df_train[forecast_level[0]].unique())
    level_col.insert(0,'Select')
    
    def level_func(l):
        global level
        if l == 'Select':
            return
        else:
            level=l

hide_toggle()

In [ ]:
def time_freq(f):
    global freq
    freq = f
    
hide_toggle()

In [ ]:
if len(forecast_level) == 2:
    interact(level1_func, l1=widgets.Dropdown(options=level1_col, value='Select',description=forecast_level[0], disabled=False))
    interact(level2_func, l2=widgets.Dropdown(options=level2_col, value='Select',description=forecast_level[1], disabled=False))
    interact(time_freq, f=widgets.IntText(value=7,description='Time Freq',disabled=False))
    
if len(forecast_level) == 1:
    interact(level_func, l=widgets.Dropdown(options=level_col, value='Select',description=forecast_level[0], disabled=False))
    interact(time_freq, f=widgets.IntText(value=7,description='Time Freq',disabled=False))

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)

if len(forecast_level) == 2:
    plot_df = df_train[(df_train[forecast_level[0]]==level1) & (df_train[forecast_level[1]]==level2)]
if len(forecast_level) == 1:
    plot_df = df_train[df_train[forecast_level[0]]==level].reset_index()

plt.figure(figsize=(10,6))
x = plot_df[date_col]
y = plot_df[target_var]

plt.plot(x,y)

print('\033[1m','Times series at selected forecast level','\033[0m')
plt.show()

hide_toggle()

In [ ]:
pylab.rcParams['figure.figsize'] = (10, 6) 
result = seasonal_decompose(plot_df[target_var], model='additive', freq=freq)
result.plot()
print('\033[1m','Decomposed components of the time series are as follows','\033[0m')
plt.show()

# 2.0. Forecasting algorithms

Following are the sections with the forecasting algorithms. Run only the respective sections for which you would want forecast values. There is no need to run the other sections.

    2.1 Random Forest
    2.2 VAR
    2.3 Prophet

# 2.1. Forecasting using Random Forest

This section forecasts using Random Forest. For more inoformation on forecastingusing random forest please go through the 'References' at the beginning.

In [ ]:
def feat_col_fun(col):
    global feat_col
    if len(col) == 0:
        return
    else:
        feat_col = list(col)

hide_toggle()

**Specify predictor columns**

In [ ]:
interact(feat_col_fun,col=widgets.SelectMultiple(options=plot_df.columns,description='Predictor columns', disabled=False))

In [ ]:
predictors = plot_df[feat_col]
target = plot_df[[target_var]]
x_train,x_cv,y_train,y_cv=train_test_split(predictors,target,test_size=0.2,random_state=1)

hide_toggle()

**Hyperparameter Optimization for Random Forest Hyperparameters**

A model hyperparameter is a characteristic of a model that is external to the model and whose value cannot be estimated from data. The value of the hyperparameter has to be set before the learning process begins. Grid-search is used to find the optimal hyperparameters of a model which results in the most ‘accurate’ predictions.

The following blocks might take some time to run.
Please specify other sets of hyperparameters if required in the following block. 

In [ ]:
#### specifying hyperparameters
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 200, num = 4)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'random_state':[1234]}

hide_toggle()

**Viewing the best set of hyperparameters**

In [ ]:
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 0)

hide_toggle()

In [ ]:
# Fit the grid search to the data
import warnings
warnings.filterwarnings("ignore")

grid_search.fit(x_train, y_train)

hide_toggle()

In [ ]:
print('\033[1m','The optimal parameters are:','\033[0m'),grid_search.best_params_

**Model training on optimized hyperparameters**

In [ ]:
optimized_rf_model = grid_search.best_estimator_
pred=optimized_rf_model.predict(x_cv)
error_rf = get_error_metrics(y_cv[target_var],pred,'Random_Forest')

hide_toggle()

In [ ]:
print('\033[1m','The in-sample error metrics are as follows:','\033[0m','\n'),print(error_rf)

hide_toggle()

In [ ]:
rf_predict = pd.DataFrame(optimized_rf_model.predict(predictors),columns=['Prediction']).reset_index()
actual = plot_df[[target_var]].reset_index()
df = pd.concat([rf_predict,actual],axis=1)
df = df.drop(['index'], axis=1)

hide_toggle()

**Plot of actuals vs forecasted values**

In [ ]:
df.plot()

# 2.2. Forecasting using Vector Autoregression 

In [ ]:
def var_func(v):
    global var_col
    if len(v) == 0:
        return
    else:
        var_col = list(v)
    
hide_toggle()

**Select time series columns to be used in VAR**

In [ ]:
var_col_options = plot_df.columns
var_col_options = var_col_options.drop([target_var,date_col])
interact(var_func,v=widgets.SelectMultiple(options=var_col_options,description='VAR columns',disabled=False))

In [ ]:
var_select_col = list([date_col])+list([target_var])+var_col
var_df = plot_df[var_select_col]
var_df = var_df.set_index(date_col)

hide_toggle()

**Testing Causation using Granger’s Causality Test**

VAR is based on the principle that each of the time series in the system influences each other. That is, one can predict the series with past values of itself along with other series in the system. Granger's causality test is used to test this relationship before building the model. 

Granger’s causality tests the null hypothesis that the coefficients of past values in the regression equation is zero. Or in simpler terms, the predictor variables do not cause the response variable. If the p-value obtained is less than the significane level of 0.05, one can safely reject the null hypothesis.

The rows in the Granger's causation matrix are the response variables and the columns are the predictor variables. The values are the p-values of the corresponding predictor variable and response variable. If the p-value is less than 0.05, the corresponding predictor variable causes the corresponding response variable.   

In [ ]:
maxlag=12
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, target_col, test='ssr_chi2test', verbose=False): 
    global min_p_dict
    i = 1
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    min_p_dict = dict()
    for c in df.columns:
        #print(c)
        for r in df.index:
            #print(r)
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print('Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
            if r == target_col:
                if min_p_value < 0.05:
                    key_name = r+'_'+str(i)
                    min_p_dict[key_name]= c, min_p_value
                    i = i+1
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df.head(1)

hide_toggle()

**Viewing the causation matrix**

In [ ]:
try:
    gcm = grangers_causation_matrix(var_df, variables = var_df.columns, target_col=target_var)
    print('\033[1m','Grangers causation matrix','\033[0m')
    print(gcm)
except:
    print('\033[1m','Please reselect other/less number of predictor columns','\033[0m')

**Selecting best causation variable to predict the time series**

In [ ]:
if len(min_p_dict)==0:
    print('Selected predictor(s) do not cause',target_var,'Please select a different set of predictor(s)')

else:
    col_list = list(var_df.columns)
    col_list.remove(target_var)
    final_list = list()
    for i in range(len(min_p_dict)):
        print(min_p_dict[list(min_p_dict.keys())[i]][0],'causes',list(min_p_dict.keys())[i],'with p-value of',min_p_dict[list(min_p_dict.keys())[i]][1])
        final_list.append(min_p_dict[list(min_p_dict.keys())[i]][0])
    print('\n')
    for i in final_list:
        print(i,'should be used as a predictor variable for',target_var)

In [ ]:
print('Selecting the following as predictor columns',final_list)
var_select_col = list([date_col])+list([target_var])+final_list
var_df = plot_df[var_select_col]
var_df = var_df.set_index(date_col)

hide_toggle()

In [ ]:
nrow = len(plot_df.index)
nobs = int(round(nrow*.80,0))-1
df_train, df_test = var_df[:nobs],var_df[nobs:]

hide_toggle()

**Checking for stationarity of times series using Augmented Dickey-Fuller Test (ADF Test)**

In order to run VAR, the times series needs to be stationary, that is, the mean and variance of the series should be constant throughout time. In case the series is not stationary using the ADF test, the times series needs to be differenced subsequently to get a stationary series. 

For a detailed explanation on ADF test click on this [link](https://www.machinelearningplus.com/time-series/augmented-dickey-fuller-test/)

In [ ]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 

    # Print Summary
    print('Augmented Dickey-Fuller Test on',name, "\n   ", '-'*47)
    print('Null Hypothesis: Data has unit root. Non-Stationary.')
    print('Significance Level is', signif)
    print('Test Statistic is' , output["test_statistic"])
    print('No. Lags Chosen is', output["n_lags"])

    for key,val in r[4].items():
        print('Critical value',key, 'is', round(val, 3))

    if p_value <= signif:
        print(" => P-Value is",p_value,"hence, rejecting Null Hypothesis.")
        print(" => Series is Stationary.")
    else:
        print(" => P-Value is",p_value,"hence, weak evidence to reject the Null Hypothesis.")
        print(" => Series is Non-Stationary.")
        
hide_toggle()

In [ ]:
for name, column in df_train.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

In [ ]:
def var_diff (x):
    if x == 'Select':
        return
    else:
        global diff
    
        diff=x
        
hide_toggle()

**Select differencing as 'Yes' if any of the time series is not stationary**
Differencing is a method of transforming a time series dataset. It can be used to remove the series dependence on time, so-called temporal dependence. Differencing can help stabilize the mean of the time series by removing changes in the level of a time series, and so eliminating (or reducing) trend and seasonality. Differencing is performed by subtracting the previous observation from the current observation.

difference(t) = observation(t) - observation(t-1)


In [ ]:
interact(var_diff,x=widgets.Dropdown(options=['Select','Yes','No'],description='Differencing?', disabled=False))

In [ ]:
if diff == 'Yes':
    df_differenced = df_train.diff().dropna()
if diff == 'No':
    df_differenced = df_train

hide_toggle()

**Selecting order of VAR model**

The order of the VAR model selected should be the one with lowest AIC and BIC values.

In [ ]:
model = VAR(df_differenced)


x = model.select_order(maxlags=6)
x.summary()

**Specify order of VAR**

In [ ]:
def var_order (o):
    global lag_order
    
    lag_order = int(o)
    
hide_toggle()

In [ ]:
interact(var_order, o=widgets.BoundedIntText(value=2,min=0,max=6,step=1,description='Order of VAR',disabled=False))

In [ ]:
model_fitted = model.fit(lag_order)

# Input data for forecasting
forecast_input = df_differenced.values[-lag_order:]

# Forecast after differencing 
fc = model_fitted.forecast(y=forecast_input, steps=df_test.shape[0])
df_forecast = pd.DataFrame(fc, index=df_test.index, columns=var_df.columns)

hide_toggle()

In [ ]:
### Invert transformation for de-differencing

def invert_transformation(df_train, df_forecast, second_diff=False):
    """Revert back the differencing to get the forecast to original scale."""
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            df_fc[str(col)+'_forecast'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)].cumsum()
        # Roll back 1st Diff
        df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)].cumsum()
    return df_fc

hide_toggle()

In [ ]:
if diff=='Yes':
    df_results = invert_transformation(df_train, df_forecast, second_diff=False)
if diff=='No':
    df_results = df_forecast

hide_toggle()

In [ ]:
var_predict = df_results[target_var].reset_index()
var_predict.rename(columns = {target_var:target_var+'_forecast'}, inplace = True)
actual = df_test[[target_var]].reset_index()
df = pd.concat([var_predict,actual],axis=1)
df = df.drop([date_col], axis=1)

hide_toggle()

**Error Metrics**

In [ ]:
error_var = get_error_metrics(df_test[target_var],df_results[target_var].values,'VAR')

print('\033[1m','The out-of-sample error metrics are as follows:','\033[0m','\n'),print(error_var)

**Plot of actuals vs forecast values**

In [ ]:
### plot
df[str(target_var)+'_forecast'].plot(legend=True).autoscale(axis='x',tight=True)
df[str(target_var)][-nobs:].plot(legend=True)

# 2.3. Forecasting using Prophet

Prophet is an open source forecasting tool by Facebook which run on an additive regression model with four main components:

* A piecewise linear or logistic growth curve trend. Prophet automatically detects changes in trends by selecting changepoints from the data.
* A yearly seasonal component modeled using Fourier series.
* A weekly seasonal component using dummy variables.
* A user-provided list of important holidays.

Prophet is mainly used to overcome the complexity introduced by variety of forecasting problems and getting reliable forecasts from that. 
The forecasts produced by prophet with default parameters is often considered as reliable as those produced by a skilled data scientist.
Although this framework of forecasting is business use-case oriented, in order to completely take leverage of the capabilities of Prophet, 
the user should make sure the observed data has the following characteristics:

* Hourly, daily, or weekly observations with at least a few months (preferably a year) of history
* Strong multiple “human-scale” seasonalities: day of week and time of year
* Important holidays that occur at irregular intervals that are known in advance
* A reasonable number of missing observations or large outliers
* Historical trend changes, for instance due to product launches or logging changes
* Trends that are non-linear growth curves, where a trend hits a natural limit or saturates

For further information visit [this](https://facebook.github.io/prophet/) link.

In [ ]:
## Required functions
def feat_col_fun2(col):
    global feat_col_pro
    if len(col) == 0:
        return
    else:
        feat_col_pro = list(col)

def seasonality(a):
    global seasonality
    if len(a) == 0:
        return
    else:
        seasonality = a
        
hide_toggle()

**Specify predictor columns**

In [ ]:
interact(feat_col_fun2,col=widgets.SelectMultiple(options=plot_df.columns,description='Predictor columns', disabled=False))

**Specify Seasonality**

In time series data, seasonality is the presence of variations that occur at specific regular intervals less than a year, such as weekly, monthly, or quarterly. Seasonality may be caused by various factors, such as weather, vacation, and holidays and consists of periodic, repetitive, and generally regular and predictable patterns in the levels of a time series.

In [ ]:
interact(seasonality,a=widgets.Dropdown(options=['Weekly','Daily','Yearly','Monthly'],description='Seasonality', disabled=False))

In [ ]:
train_dataset= pd.DataFrame()
train_dataset['ds'] = pd.to_datetime(plot_df[date_col])
train_dataset[feat_col_pro] = plot_df[feat_col_pro]
train_dataset['y']=plot_df[[target_var]]

hide_toggle()

**Train-Test Split**

Time series is intrinsically ordered dataset. The train test split in this case has to be ordered. Here we are considering the last 20 % data points as the test data set.

In [ ]:
nrow = len(plot_df.index)
a = int(round(nrow*.80,0))-1
train_X = train_dataset[:a]
test_X = train_dataset[a:]

hide_toggle()

**Model fitting & Forecasting**

In [ ]:
import warnings
warnings.filterwarnings("ignore")

if seasonality == 'Weekly':
    prophet_model = Prophet(weekly_seasonality = True)
    for i in feat_col_pro:
        prophet_model.add_regressor(i)
elif seasonality == 'Yearly':
    prophet_model = Prophet(yearly_seasonality = True)
    for i in feat_col_pro:
        prophet_model.add_regressor(i)
elif seasonality == 'Monthly':
    prophet_model = Prophet()
    prophet_model = prophet_model.add_seasonality(name = 'monthly',fourier_order = 5,period = 12)
    for i in feat_col_pro:
        prophet_model.add_regressor(i)
else:
    prophet_model = Prophet(daily_seasonality = True)
    for i in feat_col_pro:
        prophet_model.add_regressor(i)

#Fitting the data
prophet_model.fit(train_X)
#b = len(test_X.index)
#future_data = prophet_model.make_future_dataframe(periods=b , include_history = True)
#forecast the data for Test  data
forecast_data = prophet_model.predict(train_dataset)
#prophet_model.plot(forecast_data)

hide_toggle()

**Error Metrics**

In [ ]:
error_prophet = get_error_metrics(test_X['y'],forecast_data.yhat[len(train_X.index):],'Prophet')

print('\033[1m','The out-of-sample error metrics are as follows:','\033[0m','\n'),print(error_prophet.to_string(index= False))

**Goodness of Fit**

In [ ]:
fig = make_subplots(rows=1, cols=1,
                   subplot_titles = "Actual vs Predicted")

# plotting the actual train graph
fig.add_trace(go.Scatter(x= train_X['ds'], y= train_X['y'], name= "Actual : Train",
                         line_color= 'deepskyblue', mode = 'lines', opacity= 0.8,
                         legendgroup="Actual vs Predicted", showlegend=True, visible=True),
              row = 1, col = 1)

# plotting the predicted train graph
fig.add_trace(go.Scatter(x= train_X['ds'], y= forecast_data['yhat'], name= "Predicted : Train",
                         line_color= 'dimgrey', mode = 'markers', opacity= 0.8,
                         legendgroup="Actual vs Predicted", showlegend=True, visible=True),
              row = 1, col = 1)

# plotting the actual validation graph
fig.add_trace(go.Scatter(x= test_X['ds'], y= test_X['y'], name= "Actual : Validation",
                         line_color= 'brown', mode = 'lines', opacity= 0.8,
                         legendgroup="Actual vs Predicted", showlegend=True, visible=True),
              row = 1, col = 1)

# plotting the predicted validation graph
fig.add_trace(go.Scatter(x= forecast_data[a:]['ds'], y= forecast_data[a:]['yhat'], name= "Predicted : Validation",
                         line_color= 'tomato', mode = 'markers', opacity= 0.8,
                         legendgroup="Actual vs Predicted", showlegend=True, visible=True),
              row = 1, col = 1)

# add upper limit first
fig.add_trace(go.Scatter(x= forecast_data["ds"], y=forecast_data["yhat_upper"],
                         line_color= 'grey', mode = 'lines', opacity= 0.2, showlegend = False),
              row = 1, col = 1)

#finally add lower limit`
fig.add_trace(go.Scatter(x=forecast_data["ds"], y=forecast_data["yhat_lower"],
                        mode='lines',line_color= 'grey',opacity =0.2,showlegend = False,
                        fill = "tonexty", fillcolor="rgba(0,40,100,0.2)"),
              row = 1, col = 1)

# 3. Comparison of Error Metrics for the three Models

The model is passed through several metrics and a report is generated. Additional validation may be necessary depending on the model performance, the need to further diagnosis the cause of low performance, alternative metrics to explore, and analysis to confirm or explore suspicous model results. 

Compare the performance of the algorithms and select the best algorithm to save the forecast results in the next step.

In [ ]:
algo_list = list()
result = pd.DataFrame()

try:
    error_rf
    result = result.append([error_rf])
    algo_list.append('Random Forest')
except:
    print('Random Forest was not executed.')

try:
    error_var
    result = result.append([error_var])
    algo_list.append('VAR')
except:
    print('VAR was not executed.')
    
try:
    error_prophet
    result = result.append([error_prophet])
    algo_list.append('Prophet')
except:
    print('Prophet was not executed.')

hide_toggle()

In [ ]:
result

# 4. Saving Forecast Results

**For saving the forecast results user has two options:**
1. Save the Test Data Forescasts(20% of the raw data uploaded)
2. Predict on new set : If teh user has a different dataset to predict the resuls, that must be provoded in the next blocks to predict and save the forecast results

**Specify the option in the following blocks**

In [ ]:
def model_select(b):
    global model_select
    if len(b) == 0:
        return
    else:
        model_select = b

def fc_method(z):
    global fb_method_select
    if len(z) == 'Select':
        return
    else:
        fb_method_select = z

def n_steps_ahead (o):
    global n_steps_ahead

    n_steps_ahead = int(o)

fc_method_list = ['Save Test Data Forecasts','Predict on New Set']

hide_toggle()

**Specify the selected model**

In [ ]:
interact(model_select,b=widgets.Dropdown(options=algo_list,description='Model', disabled=False))

**Specify the forecast result saving method**

In [ ]:
interact(fc_method,z=widgets.Dropdown(options=fc_method_list,description='Methods', disabled=False))

**Specify the numbers of steps ahead you wish to forecast**

In [ ]:
interact(n_steps_ahead, o=widgets.BoundedIntText(value=10,min=5,max=15,step=1,description='Steps Ahead',disabled=False))

In [ ]:
def file_path_func2(file2):
    global file_path2
    
    file_path2 = file2
    
hide_toggle()

**Enter Validation csv file path in the below cell (Leave blank if method selected above is : 'Save Test Data Forecasts'**

Please make sure the csv file has the same column names as the train data and a date column in the same format as before

In [ ]:
interact(file_path_func2, file2=widgets.Text(description='Enter file path', disabled=False))

In [ ]:
if fb_method_select == 'Predict on New Set':
    fc_df = pd.read_csv(file_path2)
else:
    pass

hide_toggle()

**Writing forecasted values as csv file**

The following block will save the results based on the preferred algorithm in the previous blocks

In [ ]:
if fb_method_select =='Save Test Data Forecasts':
    if model_select == 'Random Forest':
        output = pd.DataFrame(pred,columns = [target_var+str('_forecast')])
        if len(forecast_level) == 2:
            output.insert(0,forecast_level[0], level1)
            output.insert(1,forecast_level[1], level2)
        if len(forecast_level) == 1:
            output.insert(0,forecast_level[0], level)
        output.to_csv('Forecast.csv')
        
    elif model_select == 'VAR':
        output = pd.DataFrame(df_results[target_var+str('_forecast')])
        if len(forecast_level) == 2:
            output.insert(0,forecast_level[0], level1)
            output.insert(1,forecast_level[1], level2)
        if len(forecast_level) == 1:
            output.insert(0,forecast_level[0], level)
        output.to_csv('Forecast.csv')
        
    elif model_select == 'Prophet':
        output = pd.DataFrame(forecast_data.yhat[len(train_X.index):]).rename(columns = {'yhat':target_var+str('_forecast')},inplace = False)
        if len(forecast_level) == 2:
            output.insert(0,forecast_level[0], level1)
            output.insert(1,forecast_level[1], level2)
        if len(forecast_level) == 1:
            output.insert(0,forecast_level[0], level)
        output.to_csv('Forecast.csv')
    
    
elif fb_method_select =='Predict on New Set':
    fc_df[date_col] = pd.to_datetime(fc_df[date_col],format = date_format)
    
    if model_select == 'Random Forest':
        valid_cv = fc_df[feat_col]
        pred=optimized_rf_model.predict(valid_cv)
        output = pd.DataFrame(pred,columns = [target_var+str('_forecast')])
        if len(forecast_level) == 2:
            output.insert(0,forecast_level[0], level1)
            output.insert(1,forecast_level[1], level2)
        if len(forecast_level) == 1:
            output.insert(0,forecast_level[0], level)
        output.to_csv('Forecast.csv')
    
    
    elif model_select == 'VAR':
        if diff == 'Yes':
            df_differenced = var_df.diff().dropna()
        if diff == 'No':
            df_differenced = var_df
        model = VAR(df_differenced)
        model_fitted = model.fit(lag_order)
        forecast_input = df_differenced.values[-lag_order:]
        fc = model_fitted.forecast(y=forecast_input, steps=n_steps_ahead)
        df_forecast = pd.DataFrame(fc, index=fc_df.index, columns=var_df.columns)
        if diff=='Yes':
            df_results = invert_transformation(df_train, df_forecast, second_diff=False)
        if diff=='No':
            df_results = df_forecast
        output = df_results[target_var].reset_index()
        output.rename(columns = {target_var:target_var+'_forecast'}, inplace = True)
        if len(forecast_level) == 2:
            output.insert(0,forecast_level[0], level1)
            output.insert(1,forecast_level[1], level2)
        if len(forecast_level) == 1:
            output.insert(0,forecast_level[0], level)
        output.to_csv('Forecast.csv')
    
    
    elif model_select == 'Prophet':
        valid_cv = fc_df[feat_col_pro]
        valid_cv['ds'] = pd.to_datetime(fc_df[date_col],format = date_format)
        valid_cv[feat_col_pro] = fc_df[feat_col_pro]
        valid_cv['y']=fc_df[[target_var]]
        if seasonality == 'Weekly':
            prophet_model = Prophet(weekly_seasonality = True)
            for i in feat_col_pro:
                prophet_model.add_regressor(i)
        elif seasonality == 'Yearly':
            prophet_model = Prophet(yearly_seasonality = True)
            for i in feat_col_pro:
                prophet_model.add_regressor(i)
        elif seasonality == 'Monthly':
            prophet_model = Prophet()
            prophet_model = prophet_model.add_seasonality(name = 'monthly',fourier_order = 5,period = 12)
            for i in feat_col_pro:
                prophet_model.add_regressor(i)
        else:
            prophet_model = Prophet(daily_seasonality = True)
            for i in feat_col_pro:
                prophet_model.add_regressor(i)
        prophet_model.fit(train_dataset)
        forecast_data = prophet_model.predict(valid_cv)
        output = pd.DataFrame(forecast_data.yhat).rename(columns = {'yhat':target_var+str('_forecast')},inplace = False)
        if len(forecast_level) == 2:
            output.insert(0,forecast_level[0], level1)
            output.insert(1,forecast_level[1], level2)
        if len(forecast_level) == 1:
            output.insert(0,forecast_level[0], level)
        output.to_csv('Forecast.csv')

hide_toggle()

In [ ]:
print('\033[1m','First 5 rows of the output dataset','\033[0m')
output.head()